# Ƭ̵̬̊  Why Lyric Analysis Ƭ̵̬̊
The complexity of lyrics is declining. If the words used in songs are compared to reading levels, the reading level is that of elementary students. Or at least some [studies](https://www.theguardian.com/music/2015/may/19/new-study-reveals-lyric-intelligence-getting-lower) suggest.

Inspired by this study, I decided to see if it was possible to use text mining, NLP, machine learning, and other methods to find if these studies hold some merit.

## Ƭ̵̬̊  Questions Ƭ̵̬̊
For this project, I came up with a few questions:
1. Are there particular themes that are more appealing to society due to how highly a song is rated?
2. Is it possible to predict how well a song will be received based solely on lyrical analysis?
3. Which artists are considered more "lyrically inelligent"?
4. Does lyrical intelligence depend on the genre of music?

We will examine the complexity of Ol' Blue Eyes's (Frank Sinatra's) music.

In [1]:
# first we will scrape Billboard Chart information
# and lyrics from various sites
# match information under song title
import pandas as pd
import numpy as np
from plotnine import *
from bs4 import BeautifulSoup as bs
import requests
import re
#import urllib.request
#import urllib.parse

In [2]:
# connect to the site using Beautifulsoup
source = None
url = 'https://www.billboard.com/music/frank-sinatra/chart-history/TLP'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}

try:
    source = requests.get(url, headers=headers) # urlreq.urlopen(url).read()
    source.content.decode()
    
except Exception as e:
    print(e)

In [3]:
# status code 200 means page is downloaded successfully
source

<Response [200]>

-------------------------------------
We are only interested in the following tag classes for extracting information about the song:

-------------------------------------

In [11]:
# classes for extracting information
class_album_cover = 'chart-history__item__image'
# p
class_song = 'chart-history__titles__list__item__title'
class_artist = 'chart-history__titles__list__item__artist'
class_peak = 'chart-history__titles__list__item__peak'
#a
class_peak_date = 'color--secondary font--bold'

In [14]:
parse_song_list = 'chart-history__titles__list__item'

soup = bs(source.content, 'html.parser')
find_list = soup.find_all('ul', class_="chart-history__titles__list")
# find_list # uncomment to see parsed html code

[<ul class="chart-history__titles__list">
 <li class="chart-history__titles__list__item">
 <div class="chart-history__item">
 <a class="chart-history__item__image" href="/music/frank-sinatra/chart-history/TLP/song/569348" style="background-image: url('https://charts-static.billboard.com/img/2008/05/frank-sinatra-ytu-nothing-but-the-best-isv.jpg');"></a>
 <a class="chart-history__item__left" href="/music/frank-sinatra/chart-history/TLP/song/569348">
 <p class="chart-history__titles__list__item__title color--primary font--semi-bold">Nothing But The Best</p>
 <p class="chart-history__titles__list__item__artist">Frank Sinatra</p>
 </a>
 <div class="chart-history__item__right">
 <p class="chart-history__titles__list__item__peak">
 <span class="font--semi-bold">Peaked</span> at #2 on
 <a class="color--secondary font--bold" href="">31.05.2008</a>
 </p>
 </div>
 </div>
 </li>
 <li class="chart-history__titles__list__item">
 <div class="chart-history__item">
 <a class="chart-history__item__imag

In [17]:
# create a csv file to save song info
filename = "Strangers_of_the_Night.csv"
f = open(filename, "w")

try:
    soup = bs(find_list.__str__(), 'html.parser')
    containers = soup.find_all('li')
    
    file_header = "Billboard Number, Date, Song Title, Artist\n"
    f.write(file_header)
    
    for container in containers:
        # find song peak
        get_peak = container.find("p", {"class": class_peak})
        peak = get_peak.get_text().strip()

        # extract ranking from sentence
        res = [float(s) for s in re.findall(r'-?\d+\.?\d*', peak)] 
        peak = str(int(res[0]))

        # find date of peak
        get_date = container.find("a", {"class": class_peak_date})
        on_date = get_date.get_text().strip()

        # find artist name
        get_artist = container.find("p", {"class": class_artist})
        artist = get_artist.get_text().strip()

        # find song name
        get_song = container.find("p", {"class": class_song})
        song = get_song.get_text().strip()

#         print(f' Peak: {peak}\n Date: {on_date}\n Song: {song}\n Artist: {artist}\n')
        f.write(peak + "," + on_date + "," + song + "," + artist.replace("," , "|") + '\n')

except Exception as e:
    print(str(e))
    
f.close()
print("Finished writing to file!")

Finished writing to file!
